In [26]:
import tensorflow as tf
import pandas as pd
import nltk


import csv

In [81]:

#Organizando a base de dados em um arquivo csv
#
df_txt = pd.read_csv('baseCompleta.txt', delimiter='\t', header=None)

nome_arquivo_csv = "baseCompleta.csv"

with open(nome_arquivo_csv, mode='w', newline='',encoding='utf-8') as arquivo_csv:
    escritor_csv = csv.writer(arquivo_csv, delimiter=',', quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
    # Escrever as linhas no arquivo CSV
    for index, row in df_txt.iterrows():
        texto = row[0].replace("(", "").replace(")", "").replace("'", "")

        posicao_ultima_virgula = texto.rfind(',')
        if posicao_ultima_virgula != -1:
            dados = texto[:posicao_ultima_virgula] + texto[posicao_ultima_virgula+1:]
            if dados.count(",") >= 2:
              dados = dados.replace(",", "", 1)
            print(dados)
        alterar_ultima_virgula =dados.rfind(',')   
        
        escritor_csv.writerow(dados.split(','))
#print(f"Arquivo CSV '{nome_arquivo_csv}' criado com sucesso!")


este trabalho e agradavel,alegria
gosto de ficar no seu aconchego,alegria
fiz a adesao ao curso hoje,alegria
eu sou admirada por muitos,alegria
adoro como voce e,alegria
adoro seu cabelo macio,alegria
adoro a cor dos seus olhos,alegria
somo tao amaveis um com o outro,alegria
sinto uma grande afeto por ele,alegria
quero agradar meus filhos,alegria
me sinto completamente amado,alegria
eu amo voce,alegria
que grande alivio,alegria
a dor esta amenizando finalmente,alegria
acho que me apaixonei,alegria
amar e maravilhoso,alegria
estou me sentindo muito animada,alegria
me sinto muito bem hoje,alegria
como o luar e belo,alegria
o dia esta muito bonito,alegria
nossa como sou afortunado,alegria
as maravilhas do mundo,alegria
recebi muito carinho hoje do meus colegas,alegria
estou me sentindo reconfortada hoje,alegria
e muito bom estar com os amigos,alegria
estou muito contente com o resultado dos testes,alegria
essa pintura esta bem brilhante,alegria
temos agua em abundancia,alegria
que roupa d

In [82]:
df = pd.read_csv('baseCompleta.CSV', delimiter=',')
df

ParserError: Error tokenizing data. C error: Expected 2 fields in line 743, saw 4
